In [1]:
import geopandas as gpd
import pandas as pd
import geopandas
import shapely.geometry as geom
from polygon_geohasher.polygon_geohasher import polygon_to_geohashes, geohashes_to_polygon
import pygeohash as gh
%matplotlib inline
import geohash
import geopandas as gp
import pandas as pd
import math
import geojson
from geojson import MultiLineString
from shapely.geometry import MultiLineString
from shapely.geometry import MultiPoint
import numpy as np
from shapely import geometry
from shapely.geometry import Point, Polygon, box,LineString
from geopandas import datasets, GeoDataFrame, read_file
from geopandas.tools import overlay
from matplotlib import pyplot
import matplotlib.pyplot as plt
from shapely.geometry.polygon import LinearRing, Polygon
from numpy import cos,sin,arccos
import numpy as np
from pylab import *
from shapely.ops import cascaded_union
from shapely.geometry import Point, LineString, mapping
import scipy.interpolate
import geopy
import geopy.distance
import time
import networkx as nx
from sklearn.model_selection import train_test_split
import datetime as dt

In [9]:
processed_geohash = pd.read_csv('processed_geohash.csv')

In [10]:
processed_geohash['geohash_source'] = processed_geohash['geohash_source'].astype(str).str[:6]
processed_geohash['geohash_dest'] = processed_geohash['geohash_dest'].astype(str).str[:6]

In [19]:
processed_geohash['timestamp_source'] = pd.to_datetime(processed_geohash['timestamp_source'])
processed_geohash['timestamp_dest'] = pd.to_datetime(processed_geohash['timestamp_dest'])

In [20]:
processed_geohash.head()

,trj_id,geohash_source,geohash_dest,timestamp_source,timestamp_dest,time_taken,hour,day
0,100,w21zth,w21zth,2019-04-16 02:44:15,2019-04-16 02:44:22,7.0,2,1
1,100,w21zth,w21zth,2019-04-16 02:44:22,2019-04-16 02:44:32,10.0,2,1
2,100,w21zth,w21zth,2019-04-16 02:44:32,2019-04-16 02:44:37,5.0,2,1
3,100,w21zth,w21zth,2019-04-16 02:44:37,2019-04-16 02:44:41,4.0,2,1
4,100,w21zth,w21zt5,2019-04-16 02:44:41,2019-04-16 02:44:50,9.0,2,1


In [21]:
processed_geohash.dtypes

trj_id                       int64
geohash_source              object
geohash_dest                object
timestamp_source    datetime64[ns]
timestamp_dest      datetime64[ns]
time_taken                 float64
hour                         int64
day                          int64
dtype: object

In [23]:
main_arr = []
initial_trj_id = ''
initial_geohash = ''
initial_timestamp = ''
time_taken = 0
for row in processed_geohash.iterrows():
    # if current trip id is the same as the previous id, check for difference in geohash
    if row[1]['trj_id'] == initial_trj_id:
        if row[1]['geohash_source'] != initial_geohash:
            temp_arr = []
            temp_arr.append(row[1]['trj_id'])
            temp_arr.append(initial_geohash)
            temp_arr.append(row[1]['geohash_dest'])
            temp_arr.append(initial_timestamp)
            temp_arr.append(row[1]['timestamp_dest'])
            #print(initial_timestamp, 'initial_timestamp')
            #print(row[1]['datetime'], 'row datetime')
            #print(row[1]['datetime'] - initial_timestamp, 'difference')
            temp_arr.append((row[1]['timestamp_dest'] -initial_timestamp).total_seconds())
            temp_arr.append(row[1]['hour'])
            temp_arr.append(row[1]['day'])
            
            initial_timestamp = row[1]['timestamp_source']
            initial_geohash = row[1]['geohash_source']
            
            main_arr.append(temp_arr)
            
        
    elif row[1]['trj_id'] != initial_trj_id:
        initial_trj_id = row[1]['trj_id']
        initial_timestamp = row[1]['timestamp_source']
        initial_geohash = row[1]['geohash_source']

In [24]:
view = pd.DataFrame(main_arr)
view.head(20)

,0,1,2,3,4,5,6,7
0,100,w21zth,w21zt5,2019-04-16 02:44:15,2019-04-16 02:44:53,38.0,2,1
1,100,w21zt5,w21zsg,2019-04-16 02:44:50,2019-04-16 02:48:14,204.0,2,1
2,100,w21zsg,w21zsf,2019-04-16 02:48:06,2019-04-16 02:48:42,36.0,2,1
3,100,w21zsf,w21zsd,2019-04-16 02:48:35,2019-04-16 02:49:07,32.0,2,1
4,100,w21zsd,w21zs9,2019-04-16 02:49:03,2019-04-16 02:49:37,34.0,2,1
5,100,w21zs9,w21zs3,2019-04-16 02:49:36,2019-04-16 02:50:06,30.0,2,1
6,100,w21zs3,w21zs1,2019-04-16 02:49:59,2019-04-16 02:51:05,66.0,2,1
7,100,w21zs1,w21zs0,2019-04-16 02:50:58,2019-04-16 02:51:12,14.0,2,1
8,100,w21zs0,w21zeb,2019-04-16 02:51:10,2019-04-16 02:51:57,47.0,2,1
9,100,w21zeb,w21ze8,2019-04-16 02:51:51,2019-04-16 02:53:36,105.0,2,1


In [25]:
view.columns = ['trj_id', 'geohash_source', 'geohash_dest', 'timestamp_source', 'timestamp_dest', 'time_taken', 'hour','day']

In [26]:
view.head()

,trj_id,geohash_source,geohash_dest,timestamp_source,timestamp_dest,time_taken,hour,day
0,100,w21zth,w21zt5,2019-04-16 02:44:15,2019-04-16 02:44:53,38.0,2,1
1,100,w21zt5,w21zsg,2019-04-16 02:44:50,2019-04-16 02:48:14,204.0,2,1
2,100,w21zsg,w21zsf,2019-04-16 02:48:06,2019-04-16 02:48:42,36.0,2,1
3,100,w21zsf,w21zsd,2019-04-16 02:48:35,2019-04-16 02:49:07,32.0,2,1
4,100,w21zsd,w21zs9,2019-04-16 02:49:03,2019-04-16 02:49:37,34.0,2,1


In [27]:
view.to_csv('processed_geohash_6.csv',index=False)